In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sn

import healpy as hp
import lsst.sims.maf.metrics as metrics
import lsst.sims.maf.slicers as slicers
import lsst.sims.maf.metricBundles as metricBundles
import lsst.sims.maf.db as db



In [2]:
# load opsim database
dbpath_v15 = "/home/idies/workspace/lsst_cadence/FBS_1.5/"  # path to all opsim databases

dbpath_v17 = "/home/idies/workspace/lsst_cadence/FBS_1.7/"

dbpath_v171 = "/home/idies/workspace/lsst_cadence/FBS_1.7.1/"


# output directory
dataRawDir = '/home/idies/workspace/Temporary/lixl/scratch/outDir/tGaps/'

outDir = '/home/idies/workspace/Temporary/lixl/scratch/outDir/resultsDb/'
resultsDb = db.ResultsDb(outDir=outDir)



In [3]:
# get the name of all opsim dbs 
import glob

workpath = os.getcwd()
#workpath = '/home/idies/workspace/Storage/lixl/persistent/LSST_OpSim/unknowns'

os.chdir(dbpath_v15)  # change to opsim database directory
dblist_v15 = glob.glob('*.db') 

os.chdir(dbpath_v17)  # change to opsim database directory
dblist_v17 = glob.glob('*.db') 

os.chdir(dbpath_v171)  # change to opsim database directory
dblist_v171 = glob.glob('*.db') 

os.chdir(workpath) # change back to work directory

dblist_v15.sort()
dblist_v17.sort()
dblist_v171.sort()




In [4]:
class InterNightGapsMetric(metrics.BaseMetric):
    """
    Calculate the gap between consecutive observations between nights, in days.

    Parameters
    ----------
    reduceFunc : function, optional
       Function that can operate on array-like structures. Typically numpy function.
       Default np.median.
    """
    def __init__(self, mjdCol='observationStartMJD', nightCol='night', reduceFunc=np.median,
                 metricName='Median Inter-Night Gap', **kwargs):
        units = 'days'
        self.mjdCol = mjdCol
        self.nightCol = nightCol
        self.reduceFunc = reduceFunc
        super(InterNightGapsMetric, self).__init__(col=[self.mjdCol, self.nightCol],
                                                   units=units, metricName=metricName, **kwargs)

    def run(self, dataSlice, slicePoint=None):
        """Calculate the (reduceFunc) of the gap between consecutive nights of observations.
        Parameters
        ----------
        dataSlice : numpy.array
            Numpy structured array containing the data related to the visits provided by the slicer.
        slicePoint : dict, optional
            Dictionary containing information about the slicepoint currently active in the slicer.

        Returns
        -------
        float
            The (reduceFunc) of the gap between consecutive nights of observations, in days.
        """
        dataSlice.sort(order=self.mjdCol)
        unights = np.unique(dataSlice[self.nightCol])
        if np.size(unights) < 2:
            result = self.badval
        else:
            # Find the first and last observation of each night
            firstOfNight = np.searchsorted(dataSlice[self.nightCol], unights)
            lastOfNight = np.searchsorted(dataSlice[self.nightCol], unights, side='right') - 1
            diff = dataSlice[self.mjdCol][firstOfNight[1:]] - dataSlice[self.mjdCol][lastOfNight[:-1]]
            result = self.reduceFunc(diff)
        return result


In [5]:
def pix2radec(nside, ipix, degree=True):
    """from ipix to ra/dec"""
    
    theta, phi = hp.pix2ang(nside, ipix)
    ra = phi
    dec = np.pi/2. - theta
    if degree:
        ra = np.rad2deg(ra)
        dec = np.rad2deg(dec)
    return ra, dec

def runMetric(dbname, dbpath, flt='r', ipix_ms=None, nside=16, sqlstr=None,
              mjdCol='observationStartMJD', nightCol='night', reduceFunc=np.median,
              **kwargs
             ):
    """run filterPairTGapsMetric
    dbname: name of opsim database
    fltpair: filter pair, eg: 'ri'
    ipix_ms: pixIds of footprint
    """
    opsdb = db.OpsimDatabase(dbpath+dbname)
    
    #fltpair = ['r', 'i']
    
    if ipix_ms!=None:
        # use UserPointsSlicer for minisurvey
        ra, dec = pix2radec(nside=nside, ipix=ipix_ms)
        slicer = slicers.UserPointsSlicer(ra, dec, latLonDeg=True, badval=0)
    else:
        # use HealpixSlicer 
        slicer = slicers.HealpixSlicer(nside=nside)
    
    # metric, slicer, constraint
    metric = InterNightGapsMetric( mjdCol=mjdCol, nightCol=nightCol, 
                                  reduceFunc=reduceFunc,**kwargs)
    
    sqlconstraint = 'filter in ("{}")'.format(flt)
    if sqlstr!=None:
        sqlconstraint += ' and {}'.format(sqlstr)
        
    # bundle
    newMetricSky = metricBundles.MetricBundle(metric, slicer, sqlconstraint)
    
    # group bundle
    bundleDict = {'newMetricSky':newMetricSky}
    group = metricBundles.MetricBundleGroup(bundleDict, opsdb, outDir=outDir, resultsDb=resultsDb)
    
    # run
    group.runAll()
    #group.plotAll(closefigs = False)
    return newMetricSky




In [38]:

sqlstr = 'proposalId=1'

tgaps = {}
for flt in ['u', 'g', 'r', 'i', 'z', 'y']:
    dbname = dblist_v15[5]
    dbpath = dbpath_v15
    
    metricSky = runMetric(dbname, dbpath, flt, ipix_ms=None, nside=16, sqlstr=sqlstr,
                          mjdCol='observationStartMJD', nightCol='night', 
                          reduceFunc=np.median,)
    
    arr = metricSky.metricValues.data
    arr = arr[~np.isnan(arr)] 
    tgap = np.median (arr[arr>0.1] )
    tgaps[flt] = tgap
    print(flt, tgap)


Healpix slicer using NSIDE=16, approximate resolution 219.871130 arcminutes
Querying database SummaryAllProps with constraint filter in ("u") and proposalId=1 for columns ['fieldDec', 'fieldRA', 'observationStartMJD', 'night']
Found 117746 visits
Running:  ['newMetricSky']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.
u 22.914648534806474
Healpix slicer using NSIDE=16, approximate resolution 219.871130 arcminutes
Querying database SummaryAllProps with constraint filter in ("g") and proposalId=1 for columns ['fieldDec', 'fieldRA', 'observationStartMJD', 'night']
Found 173012 visits
Running:  ['newMetricSky']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.
g 18.02438870644619
Healpix slicer using NSIDE=16, approximate resolution 219.871130 arcminutes
Querying database SummaryAllProps with constraint filter in ("r") and proposalId=1 for columns ['fieldDec', 'fieldRA', 'observationStartMJD', 'night']


In [39]:
tgaps

{'u': 22.914648534806474,
 'g': 18.02438870644619,
 'r': 7.832929375275853,
 'i': 9.043196337594054,
 'z': 11.908981292757744,
 'y': 19.984754328055715}